In [2]:
# import
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# channel list / steps
channel_list = [128, 128, 128, 128, 64]
steps = 4

## Weight Scaled Conv2

In [5]:
import math

class WSConv2d(nn.Module):
  # 입력 channel 수, 출력 channel 수, kernel 크기, stride, padding(경계 처리), scale 보정 계수
  def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, gain=2):
    super().__init__()

    # 기본 Conv2d layer 생성
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

    # Scale
    self.scale = math.sqrt(gain / (in_channels * (kernel_size ** 2)))

    # Bias를 따로 저장, conv layer에서는 bias 제거
    self.bias = self.conv.bias
    self.conv.bias = None

    # He 초기화 기준 -> weight/bias 초기화
    # conv.weight: 정규 분포 샘플링
    # bias: 모두 0으로 초기화
    nn.init.normal_(self.conv.weight)
    nn.init.zeros_(self.bias)



  # 입력값 * scale -> weight scaling 효과
  def forward(self, x):

    # bias는 channel별로 reshape후 더함
    out = self.conv(x * self.scale) + self.bias(1, -1, 1, 1) # Conv2d shape: (batch_size, out_channels, H, W)
    return out

# Pixel Norm





In [6]:
class PixelNorm(nn.Module):
  def __init__(self):
    super().__init__()
    # eps -> 분모에 사용
    self.eps = 1e-8


  # 각 픽셀마다 벡터의 크기를 1로 정규화
  # sqrt(mean+eps)로 pixel별 norm 산출
  # x를 norm으로 나눠 픽셀 벡터 크기 = 1로 정규화
  def forward(self, x):
    return x  / (1/x.size(1) * (x**2).sum(dim=1, keepdims=True)).sqrt()

# UpDownSampling

In [ ]:
class UpDownSampling(nn.Module):
  def __init__(self, size):
    super().__init__()

    # scale_factor: 배율
    self.size = size


  def forward(self, x):
    # 최근접 보간 - 해상도 전환 시 빠르고 단순한 연산 수행
    return